In [1]:
import json
import random
from keras.utils import to_categorical
from tensorflow.keras.layers import Dense, Dropout, LSTM, Bidirectional, Conv2D
import numpy
from gensim.models.fasttext import load_facebook_model
from sklearn.model_selection import train_test_split
from features import Features

from nltk.stem import WordNetLemmatizer

from nltk.tokenize import TweetTokenizer
from nltk import tokenize
import tensorflow_hub as hub
import tensorflow as tf




lemmatizer = WordNetLemmatizer()
tokenizer = TweetTokenizer()


trained_model = None
f = Features()
f.test()

Using TensorFlow backend.


features setup correctly


In [2]:

global fasttext_model

discourse = ['other', 'agreement', 'announcement', 'appreciation', 'humor', 'answer', 'elaboration', 'negativereaction',
             'question', 'disagreement']


In [3]:
def load_fasttext():
    global fasttext_model
    fasttext_model = load_facebook_model('wiki-news-300d-1M-subword.bin')

def get_data(filename):
    load_json_data = []
    count = 0
    with open(filename) as jsonfile:
        for line in jsonfile:
            jline = json.loads(line)
            load_json_data.append(jline)
            count += 1
    return load_json_data

In [4]:
def process_data(load_data):
    global errors
    global f
    count = 0
    count_no_author = 0
    count_no_title = 0
    process_data_list = []
    process_label_list = []
    for jline in load_data:
        author = None
        if 'author' in jline['posts'][0]:
            author = jline['posts'][0]['author']
        for post in jline['posts']:
            try:
                # Structure
                features = f.getStructureFeatures(jline, post['id'])
                # Content
                if 'body' in post:
                    features.append(fasttext_Vec(post['body']))
                else:
                    features.append(numpy.zeros(300))
                    count_no_title += 1
                    
                # ge66t the vector for the parent body
                features.append(fasttext_Vec(f.getParentBody(jline, post['id'])))
                # Author
                features.append(f.isSameAuthor(jline, post))
                
                if 'author' in post:
                    features.append(fasttext_Vec(post['author']))
                    
                    if author == post['author']:
                        features.append(numpy.full(300, 1.0))
                    else:
                        features.append(numpy.full(300, 0.0))
                else:
                    features.append(numpy.zeros(300))
                    features.append(numpy.zeros(300))
                    count_no_author += 1
                
                
                
                if 'title' in jline:
                    features.append(fasttext_Vec(jline['title']))
                else:
                    features.append(numpy.zeros(300))
                    
                
                    
                    
                    
                # Community
                if 'subreddit' in jline:
                    features.append(fasttext_Vec(jline['subreddit']))
                else:
                    features.append(numpy.zeros(300))
                    
                # Thread
                features += f.thread_info(jline)
                feature_nparr = numpy.array(features)
                label = discourse.index(post['majority_type'])
                process_label_list.append([label])
                process_data_list.append(feature_nparr)
            except Exception as e:
                count += 1
    print("Total exception count: " + str(count))
    print("No authors: " + str(count_no_author))
    print("No titles: " + str(count_no_title))
    process_data_list = numpy.array(process_data_list)
    process_label_list = numpy.array(process_label_list)
    print(process_data_list.shape)
    print(process_label_list.shape)
    print("done processing")
    return process_data_list, process_label_list

In [30]:

def set_model(data):
    global trained_model
    input_shape = data[0].shape
    # After trying a bunch of different methods, this one worked the best
    t_model = tf.keras.Sequential([
        
        Bidirectional(LSTM(100, input_shape=input_shape)),
        #Bidirectional(LSTM(100)),
        #Bidirectional(LSTM(64)),
        #hub_layer,
        Dropout(0.5),
        Dense(128, activation='relu'),
        Dense(64, activation='relu'),
        Dropout(0.25),
        Dense(32, activation='relu'),
        
        #tf.keras.layers.Dense(64, activation='relu'),
        tf.keras.layers.Dense(10, activation='softmax')
    ])
    t_model.compile(optimizer='rmsprop',
                    loss='categorical_crossentropy',
                    metrics=['categorical_accuracy'])
    trained_model = t_model

def train(data, labels):
    global trained_model
    checkpoint_path = 'training_1/cp.ckpt'
    set_model(data)
    cp_callback = tf.keras.callbacks.ModelCheckpoint(checkpoint_path, save_weights_only=True, verbose=1)
    trained_model.fit(data, to_categorical(labels), validation_split=0.1, epochs=10, batch_size=128, callbacks=[cp_callback])

In [6]:
def fasttext_Vec(body):
    global fasttext_model
    global lemmatizer
    global tokenizer
    tokens = tokenizer.tokenize(body)
    output = numpy.zeros(300)
    for token in tokens:
        try:
            output = numpy.add(output, fasttext_model[lemmatizer.lemmatize(token)])
        except KeyError:
            output = numpy.add(output, numpy.zeros(300))
    return output


In [7]:
global data
global labels

load_fasttext()
json_data = get_data("coarse_discourse_dump_reddit.jsonlist")
data, labels = process_data(json_data)



/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/ipykernel_launcher.py:9: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  if __name__ == '__main__':


Total exception count: 14718
No authors: 13841
No titles: 0
(101639, 18, 300)
(101639, 1)
done processing


In [31]:
train(data, labels)

Train on 91475 samples, validate on 10164 samples
Epoch 1/10
91264/91475 [============================>.] - ETA: 0s - loss: 1.2205 - categorical_accuracy: 0.5938
Epoch 00001: saving model to training_1/cp.ckpt
91475/91475 [==============================] - 38s 419us/sample - loss: 1.2203 - categorical_accuracy: 0.5939 - val_loss: 1.1338 - val_categorical_accuracy: 0.6246
Epoch 2/10
91392/91475 [============================>.] - ETA: 0s - loss: 1.0496 - categorical_accuracy: 0.6449
Epoch 00002: saving model to training_1/cp.ckpt
91475/91475 [==============================] - 35s 386us/sample - loss: 1.0493 - categorical_accuracy: 0.6450 - val_loss: 1.0444 - val_categorical_accuracy: 0.6390
Epoch 3/10
91264/91475 [============================>.] - ETA: 0s - loss: 1.0089 - categorical_accuracy: 0.6539
Epoch 00003: saving model to training_1/cp.ckpt
91475/91475 [==============================] - 36s 389us/sample - loss: 1.0089 - categorical_accuracy: 0.6539 - val_loss: 1.0288 - val_categor

In [31]:
from sklearn.feature_extraction.text import CountVectorizer

vectorizer = CountVectorizer(analyzer='word', ngram_range=(2, 2))

In [64]:
global body
def process_body(load_data):
    global vectorizer
    global f
    global body
    global labels
    count = 0
    body_t = []
    labels_t = []
    for jline in load_data:
        for post in jline['posts']:
            try:
                features = []
                b = post['body']
                p = f.getParentBody(jline, post['id'])
                label = discourse.index(post['majority_type'])
                features.append(b)
                features.append(p)
                body_t.append([b])
                labels_t.append(label)
            except Exception as e:
                count += 1
    print(count)
    print(len(body))
    body = numpy.array(body_t)
    labels = numpy.array(labels_t)
    return "Done"

14376
101981


'Done'

In [68]:
embedding = "https://tfhub.dev/google/tf2-preview/gnews-swivel-20dim/1"
global hub_layer
hub_layer = hub.KerasLayer(embedding, input_shape=[], trainable=True, dtype=tf.string)
print(body.shape)
train(body, labels, hub_layer)

(101981, 1)


ValueError: Input 0 of layer sequential_27 is incompatible with the layer: expected ndim=3, found ndim=2. Full shape received: [None, 1]

In [32]:
def set_model(data):
    global trained_model
    input_shape = data[0].shape
    # After trying a bunch of different methods, this one worked the best
    t_model = tf.keras.Sequential([
        
        Bidirectional(LSTM(100, input_shape=(len(data), 2))),
        #Bidirectional(LSTM(100)),
        #Bidirectional(LSTM(64)),
        #hub_layer,
        Dropout(0.5),
        Dense(128, activation='relu'),
        Dense(64, activation='relu'),
        Dropout(0.25),
        Dense(32, activation='relu'),
        
        #tf.keras.layers.Dense(64, activation='relu'),
        tf.keras.layers.Dense(10, activation='softmax')
    ])
    t_model.compile(optimizer='rmsprop',
                    loss='categorical_crossentropy',
                    metrics=['categorical_accuracy'])
    trained_model = t_model

def train(data, labels):
    global trained_model
    checkpoint_path = 'training_1/cp.ckpt'
    set_model(data)
    cp_callback = tf.keras.callbacks.ModelCheckpoint(checkpoint_path, save_weights_only=True, verbose=1)
    trained_model.fit(data, to_categorical(labels), validation_split=0.1, epochs=10, batch_size=128, callbacks=[cp_callback])

In [ ]:
json_data = get_data("coarse_discourse_dump_reddit.jsonlist")
process_body(json_data)